# Apache Pinot

_Seminararbeit von Nikola Braukmüller (1424783) und Tim Ebert (8559152) zum Modul "Advanced Data Management" (W3M20011) am DHBW CAS (WiSe 2020/21)._  
_Abgabedatum: 12. April 2021_  
_Dozent: Prof. Dr. Dennis Pfisterer_

# Prerequisites

## Install Pinot on Kubernetes Cluster

To install a simple Pinot setup including 3 servers and a miminal Kafka cluster on a Kubernetes cluster, follow these steps.
In our tests we used a single-node cluster on GCP with 4 CPUs and 16 GB of Memory (instance type `n1-standard-4`). We observed that this machine was well utilized during data ingestion, so we recommend using a similarly sized cluster.  
See [values.yaml](https://github.com/timebertt/adm-pinot/blob/master/pinot/values.yaml) for further configuration options, e.g. to configure a faster storage class for Pinot's PVCs (we used SSDs (`pd-ssd`) in our tests for better performance).

```bash
# create and switch to pinot namespace
kubectl create namespace pinot
kubectl config set-context --current --namespace pinot

# install our pre-built helm chart in pinot namespace using the default configuration
helm install pinot https://raw.githubusercontent.com/timebertt/adm-pinot/master/assets/pinot-0.1.0.tgz

# wait until all pinot components are ready
kubectl get pod -w
```

Once all components are ready, you can open the Pinot dashboard to see if everything is working as expected.
```
kubectl port-forward svc/pinot-controller 9000 &
```
Navigate to http://localhost:9000/ to open the Pinot dashboard.

## Python requirements

Install the Python modules required by this Jupyter notebook:

In [ ]:
!pip install -I 'kafka-python==2.0.2' 'names==0.3.0' 'pandas==1.2.3' 'requests==2.25.1'

In [1]:
# all imports
import copy, requests, json, io, re, os, shutil, fileinput, tarfile, time, csv, random, names
import pandas as pd
from kafka import KafkaProducer

# Introduction

Pinot is an open-source distributed highly-available OLAP datastore and built to serve analytical queries on realtime event data. It is developed by engineers of LinkedIn and Uber.
LinkedIn is operating Pinot clusters for realtime Online Analytical Processing. They divide their analytics applications into two main categories in their solution landscape: Internal applications and user-facing applications. Internal applications need to process large data volume (trillions of records), but higher query latencies are tolerated. On the opposite, user-facing applications are available for hundreds of millions of LinkedIn members. These applications have a very high query volume and are expected to have a lower latency.
Pinot production clusters at LinkedIn are serving tens of thousands queries per second. Overall, more than 50 analytical use cases are supported, and millions of records are ingested per second.

## Design Principles

Key requirements for Pinot include:

- high performance (low latency) query execution
- near-realtime data ingestion
- linear horizontal scalability (in terms of data size, ingestion rate and query rate)
- query flexibility to cover a wide range of analytical use cases
- high availability of data as well as components (fault tolerance)

All of these requirements influence Pinot's fundamental design principles and distributed architecture. We present, how Pinot manages to achieve these goals in the following sections by describing the core concepts and demonstrating the most important mechanisms in Pinot.

## Architecture

A Pinot cluster is comprised of multiple distributed components. Each Pinot cluster consists of a controller, one or multiple brokers and multiple servers. Pinot supports multi-tenancy out-of-the-box, as multiple brokers and servers can be assigned to serve specific tenants. A table in pinot consists of columns and rows, which are broken horizontally into shards (named segments).

Apache Helix is a generic cluster management framework which is used for automatic management of partitioned and replicated distributed systems by creating and assigning tasks. Apache Zookeeper takes care of coordination and maintenance of the overall cluster state and health. In addition, it stores information about the cluster like server locations of a segment and table schema information. The Controller embeds the Helix agent and is the driver of the cluster. To access CRUD (Create, Read, Update, Delete) Operations on logical storage resources, it provides a REST interface.

If a client wants to query data of Pinot tables, the request will be sent to the broker. It routes queries to the appropriate server instances and keeps track on the query routing tables. These routing tables consist of a mapping between segments and server, where the segments reside on. This ensures the right routing of the query to the correct segment. Segments can either consume realtime data or data can be pushed into offline segments. By default, the query load is balanced across all available servers. The broker will return one consolidated result to the client, independent from the fact whether the table is divided into realtime and offline segments.

Servers are categorized into offline and realtime servers. According to this categorization, servers in Pinot either host offline or realtime data. The responsibility of a server is defined by the table assignment strategy.

If a new realtime table is configured, the realtime server will start consuming data from the streaming source (e.g. Kafka topic). The broker will watch the consumption, detect new segments and maintain them in the query routing list. If a segment has been completed (reached a specific amount of records or was available for a specific timeframe), the controller will upload the segment to the cluster's segment store. The status of the uploaded segment changes from "consuming" to "online" and the controller will start a new consumption on the realtime server.
With batch ingestion, already existing data (e.g. in Hadoop) can be loaded to a Pinot table. 

<img src='https://raw.githubusercontent.com/timebertt/adm-pinot/master/images/Architecture.png' width="35%" height="35%">
                                                 
Image source: https://docs.pinot.apache.org/basics/architecture (accessed April, 4th 2021)

In addition to components shown in the above architectural diagram, minions can be deployed to the cluster. They leverage Apache Helix and execute tasks which are provided by the Helix Task Executor Framwork. A minion takes over tasks with intensive workloads from other components like indexing or purging data from a Pinot cluster, for example due to GDPR compliance.
The Pinot minion can also be used for Pinot's Offline Flow, which moves records from `REALTIME` tables to corresponding `OFFLINE` tables (covered later on).

## API Interface for Broker and Controller

Queries are sent to the broker's REST API (listening on port 8099 by default).
To get information about the resources of the Pinot cluster, we are accessing the controller's REST API, which is listening on port 9000.
Broker Configurations are defined in a specific `broker.conf` file. The properties define configurations like the query port for the broker or a limit for queries. The latter of which has the purpose to protect brokers and servers against queries returning very large amount of records. A query limit needs to be enabled at cluster level. In our scenario, the parameter `pinot.broker.enable.query.limit.override` is set to false, which means that the broker won't override or add a query limit when the returned record amount is larger than defined in the broker config file.

In [2]:
print("\033[1m" + "Broker: "+ "\033[0m" + json.dumps((requests.get('http://pinot-controller.pinot:9000/v2/brokers/tenants')).json(), indent=2))
print("\033[1m" + "Health of Controller: "+ "\033[0m" + requests.get('http://pinot-controller.pinot:9000/pinot-controller/admin').text)
print("\033[1m" + "Cluster: "+ "\033[0m" + json.dumps((requests.get('http://pinot-controller.pinot:9000/cluster/configs')).json(), indent=2))

Broker: {
  "DefaultTenant": [
    {
      "host": "Broker_pinot-broker-0.pinot-broker-headless.pinot.svc.cluster.local",
      "port": 8099,
      "instanceName": "Broker_pinot-broker-0.pinot-broker-headless.pinot.svc.cluster.local_8099"
    }
  ]
}
Health of Controller: GOOD
Cluster: {
  "allowParticipantAutoJoin": "true",
  "enable.case.insensitive": "false",
  "pinot.broker.enable.query.limit.override": "false",
  "default.hyperloglog.log2m": "8"
}


## Key differences to well-known database technologies

In Pinot, data ingestion is append-only. There is no possibility to modify values after ingestion by doing operations like `UPDATE` known from databases like PostgreSQL. Pinot is no replacement for databases in an operational business environment, which usually require updates to data because of the event's nature or due to data correction. For this use cases, Pinot does not fit. Instead, it can enhance use cases requiring fast analytics. However, data can still be purged after ingestion for fullfilling compliance requirements (e.g. GDPR). For this, the Minion can be used to replace entire segments, but in no case, single records can be manipulated.

Another difference of Apache Pinot compared to databases like PostgreSQL is that it doesn't support queries requiring movements of large amounts of data between the nodes, like joins. The query engine Presto can be used to join different tables in Pinot, but Presto needs to be set up additionally and is not part of Pinot.

Tables in Pinot typically have one primary time column, which is used to manage the time boundary between offline and realtime data in a hybrid table. This may sound familiar to the known concept of time series databases like Influxdb. Both databases are built to handle events with a timestamp, but the timestamp in Pinot is only strictly required for hybrid tables. In addition, Pinot is not only focused on storing timeseries of metrics, it also offers to storm string and bytes values in addition to numeric data types and date time fields. Although Influxdb also support strings to a specific extend, Pinot also offers e.g. text indexing for enhanced full text search.
Compared to the timeseries databases like Influxdb, Pinot is optimized for storing time data with a focus on append operations and queries. Update and delete operations on single records are not supported in Apache Pinot, though stream ingestion supports upserts, if a primary key has been defined in the schema.

Another key difference of Pinot in comparison to other distributed databases is the heterogeneous nature of its components. Some traditional RDBMSs like for example PostgreSQL can be scaled horizontally to form a cluster by adding more instances, that will each store and manage different partitions (shards) of the dataset. In this case, such a distributed setup is comprised of only a single stateful component, which is started on multiple machines (homogeneous distributed system).
In contrast to this, a Pinot cluster is comprised of multiple heterogeneous components (described above), which each serve a specific purpose and are only responsible for a given subtask of the entire system. For example, servers are the stateful components of Pinot, that store and query the actual dataset, while brokers are stateless components, that don't host data themselves and only serve the query frontend for the database. With this, Pinot can be seen as a heterogeneous distributed system, which makes it more complex to deploy and operate, but also serves the key requirements described above (mainly horizontal scalability and fault tolerance).

## Schemas and Tables

### Schemas

To create a table in Pinot, a schema is required. A schema configuration defines fields and data types, this metadata is stored in the Zookeeper.
In our examples, we work with data of a fictional online plattform which connects car drivers and passengers to travel together in Germany (ride sharing). 

Columns in Pinot are of different categories: 
- dimension columns: support operations like `GROUP BY` and `WHERE` ("slice and dice"), e.g. name of the car driver, trip start and end location
- metric columns: represent quantitative data and can be used e.g. for aggregation clauses (e.g. payment amount, rating of the driver)
- DateTime columns: represent timestamps of records. One DataTime column can be treated as the primary time column, which is defined in the segment config of a table. The primary time column is used for determining boundaries of segments and between offline and realtime data in hybrid tables. A typical operation on DateTime columns is for example `WHERE`, e.g. to select rides of a given day

Let's define the example `trips` schema:

In [3]:
schemaConfiguration = {
  "schemaName": "trips",
  "dimensionFieldSpecs": [
    {
      "name": "rider_name",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "driver_name",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "license_plate",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "start_location",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "start_zip_code",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
     {
      "name": "start_location_state",
      "dataType": "STRING",
      "defaultNullValue": ""
    }, 
    {
      "name": "end_location",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "end_zip_code",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
      {
      "name": "end_location_state",
      "dataType": "STRING",
      "defaultNullValue": ""
    }, 
    {
      "name": "rider_is_premium",
      "dataType": "INT",
      "defaultNullValue": 0
    }
  ],
  "metricFieldSpecs": [
    {
      "name": "count",
      "dataType": "LONG",
      "defaultNullValue": 1
    },
    {
      "name": "payment_amount",
      "dataType": "FLOAT",
      "defaultNullValue": 0
    },
    {
      "name": "payment_tip_amount",
      "dataType": "FLOAT",
      "defaultNullValue": 0
    },
    {
      "name": "trip_wait_time_millis",
      "dataType": "LONG",
      "defaultNullValue": 0
    },
    {
      "name": "rider_rating",
      "dataType": "INT",
      "defaultNullValue": 0
    },
    {
      "name": "driver_rating",
      "dataType": "INT",
      "defaultNullValue": 0
    }
  ],
  "dateTimeFieldSpecs": [
    {
      "name": "trip_start_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "PRIMARY"
    },
    {
      "name": "request_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "SECONDARY"
    },
    {
      "name": "trip_end_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "SECONDARY"
    }
  ]
}

# create the trips schema
response = requests.post('http://pinot-controller.pinot:9000/schemas?override=true', json=schemaConfiguration)
print("Create Schema: " + response.text)

# list all Schemas
response = (requests.get('http://pinot-controller.pinot:9000/schemas')).json()
print("Get all schemas: " + str(response))

Create Schema: {"status":"trips successfully added"}
Get all schemas: ['trips']


### Data Generation

Our Pinot tables will consume data from a Kafka Topic in realtime. To be able to consume messages from this topic, data needs to be produced and sent to the topic before.

To create and fill our Kafka topic, we first need to create a Kafka producer client.

In [4]:
producer = KafkaProducer(bootstrap_servers=['pinot-kafka.pinot:9092'], value_serializer=lambda v: json.dumps(v).encode('utf-8'))

The below functions are used to generate random data records for car rides in Germany and inserts them to the Kafka Topic. Each ride consists of driver and passenger details, such as name and rating, measures like payments, details about origin and destination of the trip and different time measures, for example the time stamp when the trip was requested. Date and time of the trip is generated based on the current timestamp (and advancing by roughly 1 second per record).

In [5]:
# Choose random city of file containing German cities with postcode
if not os.path.exists("./pgeocodeDE.txt"):
    # download segment to local file
    response = requests.get("https://symerio.github.io/postal-codes-data/data/geonames/DE.txt")
    with open("./pgeocodeDE.txt", 'w',encoding='utf8') as out_file:
        out_file.write(response.text)
    del response

geocode_file = open('./pgeocodeDE.txt')
geocode_list = list(csv.reader(geocode_file, delimiter='\t'))[1:] # skip first line (header)
random.shuffle(geocode_list)
geocode_list = geocode_list[:1000] # take only random 1000 places to generate more overlapping data
geocode_file.close()

def choose_random_city():
    return random.choice(geocode_list)

# generate only 1000 driver/rider names to generate more overlapping data
names_list = []
for i in range(1000):
    names_list.append(names.get_full_name())

def choose_random_name():
    return random.choice(names_list)
    
# Generation of License Plate
# create a pool of letters to choose from
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
numbers = '0123456789'

def generate_license_plate():
    # generate 3 randomly chosen letters, L1, L2, L3
    L1 = random.choice(letters)
    L2 = random.choice(letters)
    L3 = random.choice(letters)
    L4 = random.choice(letters)
    # generate 4 randomly chosen numbers, N1, N2, N3, N4
    N1 = random.choice(numbers)
    N2 = random.choice(numbers)
  
    # combine it together into one print function
    return(L1+L2+'-'+L3+L4+'-'+N1+N2)

# Calculation of price based on distance between start city and end destination
def calculate_price(v_distance):
    v_multiplicator=round(random.uniform(0.8, 2.0),2)
    v_price=round(v_distance*v_multiplicator,2)
    return(v_price)

Let's generate our sample dataset, containing about 300.000 records in total, in order to demonstrate the different Pinot concepts and mechanisms later on:

In [6]:
# begin generating trips data at current time
start_timestamp_ms = time.time_ns() // 1000000

# Generate data
num_records = 300000 + random.randint(5000,10000)
for i in range(num_records):
    v_start_location = choose_random_city()
    v_end_location = choose_random_city()
    v_distance = random.randint(5,1000)

    # add random jitter, in large system our event stream is probably also not strictly sorted
    v_requesttime = start_timestamp_ms + i*1000 + random.randint(0,100);

    v_waiting_time_millis = random.randint(1,3600000)
    v_trip_time = round((v_distance/random.randint(45,60)) * 60 *60*1000)

    record = {
        "rider_name": choose_random_name(),
        "driver_name": choose_random_name(),
        "license_plate": generate_license_plate(),
        "start_location": v_start_location[2],
        "start_zip_code": v_start_location[1],
        "start_location_state": v_start_location[3],
        "end_location": v_end_location[2],
        "end_zip_code": v_end_location[1],
        "end_location_state": v_end_location[3],
        "rider_is_premium": random.randint(0, 1),
        "count": 1,
        "payment_amount": calculate_price(v_distance),
        "payment_tip_amount": random.randint(5,50),
        "trip_wait_time_millis": v_waiting_time_millis,
        "rider_rating": random.randint(0,5),
        "driver_rating": random.randint(0,5),
        "trip_start_time_millis": v_requesttime + v_waiting_time_millis,
        "request_time_millis": v_requesttime,
        "trip_end_time_millis": v_requesttime + v_waiting_time_millis + v_trip_time
    }
 
    producer.send('trips', value=record)
        
    if i % 5000 == 0:
        print(f'{i} records generated')

print(f'done generating {num_records} records, ready to do some fancy analytics!')

0 records generated
5000 records generated
10000 records generated
15000 records generated
20000 records generated
25000 records generated
30000 records generated
35000 records generated
40000 records generated
45000 records generated
50000 records generated
55000 records generated
60000 records generated
65000 records generated
70000 records generated
75000 records generated
80000 records generated
85000 records generated
90000 records generated
95000 records generated
100000 records generated
105000 records generated
110000 records generated
115000 records generated
120000 records generated
125000 records generated
130000 records generated
135000 records generated
140000 records generated
145000 records generated
150000 records generated
155000 records generated
160000 records generated
165000 records generated
170000 records generated
175000 records generated
180000 records generated
185000 records generated
190000 records generated
195000 records generated
200000 records generated


### Tables

Tables represent a collection of related data in Pinot. A table either have the type `OFFLINE` (ingesting pre-built pinot-segments from external stores) or `REALTIME` (data ingestion from streams). The user is not required to know the type of a table when querying it.

In [7]:
# some helper functions
def query_sql(query):
    print("query: " + query)
    return requests.get('http://pinot-broker.pinot:8099/query/sql', params={
        "sql" : query,
        "trace": "true"
    }).json()

def query_result_to_dataframe(result):
    return pd.DataFrame(columns=result['resultTable']['dataSchema']['columnNames'], data=result['resultTable']['rows'])

To configure a table, properties like name, type and indexing are required. In the following example, we create an example table which is consuming data from the Kafka topic filled above:

In [8]:
table_config = {
  "tableName": "trips",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "trips",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "20000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    },
  },
  "metadata": {
    "customConfigs": {}
  }
} 

response = requests.post('http://pinot-controller.pinot:9000/tables', json=table_config)
print(response)
print(response.json())

<Response [200]>
{'status': 'Table trips_REALTIME succesfully added'}


After creation, data records of the Kafka Topic are loaded into the table. To execute a query, the SQL statement is sent to the broker of the Pinot cluster. The response contains the result records, as well as query statistics of the execution.

While our data is loading, let's query the example table to find out the top 5 states where trips of our ride sharing platform start:

In [9]:
query_result_to_dataframe(query_sql("""
    SELECT start_location_state, SUM(count) as trips_count
    FROM trips
    GROUP BY start_location_state
    ORDER BY trips_count DESC
    LIMIT 5"""
))

query: 
    SELECT start_location_state, SUM(count) as trips_count
    FROM trips
    GROUP BY start_location_state
    ORDER BY trips_count DESC
    LIMIT 5


,start_location_state,trips_count
0,Bayern,1534.0
1,Rheinland-Pfalz,1448.0
2,Niedersachsen,1423.0
3,Brandenburg,924.0
4,Sachsen-Anhalt,864.0


# Segmentation in Pinot

## Introduction

Table contents in Pinot are expected to grow infinitely and thus need to be distributed across multiple nodes. Therefore, the tables' dataset is split into segments, which are comparable to shards/partitions in classical RDBMSs. In Pinot, segmentation is done in a time-based fashion, meaning that configured timestamps of records in a given segment will be close to each other.
Segments store all columns of a table and organize data in columnar orientation for high encoding efficiency and optional pre-aggregation of metrics. In addition to the data itself, segments contain indices and other lookup-related data structures like dictionaries.

As Pinot is not a general-purpose database (data is immutable), it cannot be used as an application's "main datastore". Like other OLAP stores, Pinot is supposed to run next to the application's "main datastore" and its data has to be imported separately (ingestion). In order to facilitate near-realtime analytical queries, for example like the ones powering LinkedIn's well-known "Who viewed my profile" functionality, data is typically ingested into Pinot via event streaming platforms, like Apache Kafka (stream ingestion). In contrast to classical RDBMSs, Pinot comes with built-in support for directly reading from Kafka event streams.
However, data can also be ingested from traditional batch processing workflows, for example realized with Apache Hadoop or Apache Spark (batch ingestion).

Pinot tables are either defined as realtime or offline tables. Tables of both types are broken into segments. For realtime tables, data is consumed directly from event streams by Pinot servers as-is without any additional processing. Segments are built inside Pinot and are completed once a given threshold in size or time is reached. Segments for offline tables are built outside of Pinot in batch processing jobs, that might perform additional data deduplication or similar processing, and uploaded to the Pinot controller. Both table types might be combined to form hybrid tables, that allow both realtime analytics as well as long-term data storage (covered later on).

## Realtime Data Ingestion

To demonstrate how segments work in Pinot, we're going to focus on realtime data ingestion first. In the following examples, we'll be using the controller's and broker's REST APIs in order to dynamically create realtime tables, retrieve segment metadata and execute SQL queries.

In [10]:
# some helpers for the upcoming examples
def server_name_from_instance(instance):
    return re.search('pinot-server-[0-9]+', instance).group()

def extract_query_statistics_from_result(result):
    query_statistics_fields = ["numServersQueried","numServersResponded","numSegmentsQueried","numSegmentsProcessed","numSegmentsMatched","numConsumingSegmentsQueried","numDocsScanned","numEntriesScannedInFilter","numEntriesScannedPostFilter","numGroupsLimitReached","totalDocs","timeUsedMs"]
    return { key: result[key] for key in query_statistics_fields }

def extract_query_statistics_from_result_dataframe(result):
    return pd.DataFrame({"value": extract_query_statistics_from_result(result)})

ordinal_pattern = re.compile(r'__[0-9]+__([0-9]+)__')
def sort_by_ascending_ordinal(segments):
    segments.sort(key=lambda L: (int(ordinal_pattern.search(L).group(1)), L))

def segment_metadata_for_table(table):
    segments = requests.get(f'http://pinot-controller.pinot:9000/segments/{table}').json()
    
    segment_metadata = {}
    for segments_item in segments:
        for table_type, type_segments in segments_item.items():
            for segment in type_segments:
                segment_type_name = f"{segment}_{table_type}"
                segment_metadata[segment_type_name] = requests.get(f'http://pinot-controller.pinot:9000/segments/{table}/{segment}/metadata').json()
    
    return segment_metadata

def segment_metadata_of_nth_segment(segment_metadata, n, table_type="REALTIME"):
    segments_of_type = []
    for segment in segment_metadata.keys():
        if segment.endswith("_" + table_type):
            segments_of_type.append(segment)
    
    sort_by_ascending_ordinal(segments_of_type)
    return segment_metadata[segments_of_type[n]]


def start_time_of_nth_segment(segment_metadata, n, table_type="REALTIME"):
    return segment_metadata_of_nth_segment(segment_metadata, n, table_type)["segment.start.time"]

def wait_for_table_to_finish_loading(table, wait_time=15):
    last_total_docs = -1
    while True:
        response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={"sql" : f"SELECT * FROM {table} LIMIT 1"}).json()
        total_docs = response["totalDocs"]
        if total_docs == last_total_docs:
            print(f"--Consumption of generated data for table {table} finished, (loaded {last_total_docs} docs)--")
            break
        
        last_total_docs = total_docs
        print(f"waiting for table {table} to finish loading (loaded {last_total_docs} docs)")
        time.sleep(wait_time)

At first, we will create two realtime tables. Both will be using the `trips` schema created above and read from the `trips` topic in Kafka, that was also created and filled with random records above.

In [11]:
# common configuration used for both tables
table_config_template = {
  "tableName": "",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "invertedIndexColumns": [
        "rider_name",
        "driver_name",
        "start_location",
        "end_location"
    ],
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.topic.name": "trips",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
}

Pinot servers will continuously read from the Kafka topic into memory and compile a segment until a configured threshold is reached. The first table is configured to flush the new in-memory segment to disk, once either 12 hours have passed or the segment contains 80,000 rows (which will be the case for our example, as the data is already waiting in the Kafka stream).

In [12]:
# create first table
table_config = copy.deepcopy(table_config_template)
table_config["tableName"] = "trips_segmentation_1"
table_config["segmentsConfig"]["replication"] = "1"
table_config["segmentsConfig"]["replicasPerPartition"] = "1"
table_config["tableIndexConfig"]["streamConfigs"]["realtime.segment.flush.threshold.time"] = "12h"
table_config["tableIndexConfig"]["streamConfigs"]["realtime.segment.flush.threshold.size"] = "80000"
display(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

{'status': 'Table trips_segmentation_1_REALTIME succesfully added'}

In contrast to the first table, the second one will target a segment size of 50,000 rows and will additionally create 3 replicas of each segment on different server instances for data availability (fault tolerance) and load distribution of queries.

In [13]:
# create second table
table_config = copy.deepcopy(table_config_template)
table_config["tableName"] = "trips_segmentation_2"
table_config["segmentsConfig"]["replication"] = "3"
table_config["segmentsConfig"]["replicasPerPartition"] = "3"
table_config["tableIndexConfig"]["streamConfigs"]["realtime.segment.flush.threshold.time"] = "12h"
table_config["tableIndexConfig"]["streamConfigs"]["realtime.segment.flush.threshold.size"] = "50000"
display(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

{'status': 'Table trips_segmentation_2_REALTIME succesfully added'}

Let's wait for the tables to finish loading the data from Kafka:

In [14]:
wait_for_table_to_finish_loading("trips_segmentation_1")
wait_for_table_to_finish_loading("trips_segmentation_2")

waiting for table trips_segmentation_1 to finish loading (loaded 54899 docs)
waiting for table trips_segmentation_1 to finish loading (loaded 93568 docs)
waiting for table trips_segmentation_1 to finish loading (loaded 160000 docs)
waiting for table trips_segmentation_1 to finish loading (loaded 240000 docs)
waiting for table trips_segmentation_1 to finish loading (loaded 309911 docs)
--Consumption of generated data for table trips_segmentation_1 finished, (loaded 309911 docs)--
waiting for table trips_segmentation_2 to finish loading (loaded 309911 docs)
--Consumption of generated data for table trips_segmentation_2 finished, (loaded 309911 docs)--


The controller stores metadata for each segment, which can be viewed via its REST API. Each segment's metadata contains general information such as the table type, table name and time unit as well as segment-specific information such as the number of records (`segment.total.docs`), the timestamp of the segment's first and last record (`segment.start.time`, `segment.end.time`) and the segment's status (`segment.realtime.status`).
New realtime segments start in status `IN_PROGRESS`, which means that the segment is currently consuming data from the Kafka topic. Once the size or time threshold is reached, the consuming servers start a segment commit protocol in order to agree on the last record that shall be included in the segment. Once the commit protocol is completed, the segment transitions to `DONE` and the servers flush the data to disk. Afterwards, a new segment is started again to consume further data from the event stream.

We can now query the controller's REST API to retrieve metadata for all segments in both our tables.
The first table contains less segments, but each segment contains a higher number of records.

In [15]:
segment_metadata_1 = segment_metadata_for_table("trips_segmentation_1")
pd.DataFrame(segment_metadata_1)

,trips_segmentation_1__0__0__20210411T1645Z_REALTIME,trips_segmentation_1__0__1__20210411T1646Z_REALTIME,trips_segmentation_1__0__2__20210411T1646Z_REALTIME,trips_segmentation_1__0__3__20210411T1646Z_REALTIME
segment.realtime.endOffset,80000,160000,240000,9223372036854775807
segment.time.unit,MILLISECONDS,MILLISECONDS,MILLISECONDS,None
segment.start.time,1618159486495,1618239458972,1618319463311,-1
segment.flush.threshold.size,80000,80000,80000,80000
segment.realtime.startOffset,0,80000,160000,240000
segment.end.time,1618242914999,1618322911292,1618402828195,-1
segment.total.docs,80000,80000,80000,-1
segment.table.name,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME
segment.realtime.numReplicas,1,1,1,1
segment.creation.time,1618159544768,1618159567495,1618159588274,1618159603181


The segment metadata for the second table shows more segments. Each of them has a lower number of total records and 3 replicas (`segment.realtime.numReplicas`).

In [16]:
segment_metadata_2 = segment_metadata_for_table("trips_segmentation_2")
pd.DataFrame(segment_metadata_2)

,trips_segmentation_2__0__0__20210411T1645Z_REALTIME,trips_segmentation_2__0__1__20210411T1646Z_REALTIME,trips_segmentation_2__0__2__20210411T1646Z_REALTIME,trips_segmentation_2__0__3__20210411T1646Z_REALTIME,trips_segmentation_2__0__4__20210411T1646Z_REALTIME,trips_segmentation_2__0__5__20210411T1646Z_REALTIME,trips_segmentation_2__0__6__20210411T1646Z_REALTIME
segment.realtime.endOffset,50000,100000,150000,200000,250000,300000,9223372036854775807
segment.time.unit,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,None
segment.start.time,1618159486495,1618209432220,1618259456044,1618309433532,1618359464465,1618409462241,-1
segment.flush.threshold.size,50000,50000,50000,50000,50000,50000,50000
segment.realtime.startOffset,0,50000,100000,150000,200000,250000,300000
segment.end.time,1618212853532,1618262875514,1618312939951,1618362932066,1618412895838,1618462970400,-1
segment.total.docs,50000,50000,50000,50000,50000,50000,-1
segment.table.name,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME
segment.realtime.numReplicas,3,3,3,3,3,3,3
segment.creation.time,1618159546358,1618159564259,1618159579112,1618159591215,1618159601406,1618159611054,1618159619674


Pinot brokers are responsible for executing queries against the database. When a broker receives a new query, it sends multiple subqueries to Pinot servers that are hosting the segments belonging to the queried table. Once it has received results from all queried servers, it merges the subresults and returns the aggregated result to the client.
In order to efficiently execute queries, brokers use segment metadata to figure out, which segments need to be queried. For example, if we want to list the top 5 drivers in terms of trips count in a given timeframe, only the segments hosting data of the timeframe need to be queried.

To demonstrate this behaviour, we call the broker's REST API and query data from the time range of the first segment (before start time of the second segment). In the returned query statistics we can see, that not all segments of the table (`numSegmentsQueried`) are actually processed, but only 2 of them (`numSegmentsMatched`). This is because the last (the consuming) segment is always queried, as the metadata is not yet completed and so the broker can't tell upfront, if the last segment might contain relevant data.

In [21]:
# get data from first segment (consuming segment is always queried because of uncompleted metadata)
query_for_trips_segmentation_1 = f"""
    SELECT driver_name, sum(count) AS trips_count
    FROM trips_segmentation_1
    WHERE trip_start_time_millis BETWEEN {start_time_of_nth_segment(segment_metadata_1, 0)} AND {int(start_time_of_nth_segment(segment_metadata_1, 1))-1}
    GROUP BY driver_name
    ORDER BY trips_count desc
    LIMIT 5"""

query_result = query_sql(query_for_trips_segmentation_1)
display(query_result_to_dataframe(query_result))
display(extract_query_statistics_from_result_dataframe(query_result))

query: 
    SELECT driver_name, sum(count) AS trips_count
    FROM trips_segmentation_1
    WHERE trip_start_time_millis BETWEEN 1618159486495 AND 1618239458972
    GROUP BY driver_name
    ORDER BY trips_count desc
    LIMIT 5


,driver_name,trips_count
0,Alice Reese,161.0
1,Dennis Smith,142.0
2,Kelly Sprague,113.0
3,Darius Blanco,105.0
4,Terra Bell,104.0


,value
numConsumingSegmentsQueried,1
numDocsScanned,78268
numEntriesScannedInFilter,160000
numEntriesScannedPostFilter,156536
numGroupsLimitReached,False
numSegmentsMatched,2
numSegmentsProcessed,2
numSegmentsQueried,4
numServersQueried,1
numServersResponded,1


The second query targets the second table and lists the top 5 drivers according to rating over the time range of the first 3 segments.
Similarly to the query above, only relevant segments need to be processed for this query.
However, in contrast to the first query execution, the broker can make use of the segment replication and can distribute the subqueries for individual segments across different servers (note that `numServersQueried` is now 3 instead of 1).

In [23]:
# get data from first 3 segments (consuming segment is always queried because of uncompleted metadata)
query_for_trips_segmentation_2 = f"""
    SELECT driver_name, avg(driver_rating) AS rating
    FROM trips_segmentation_2
    WHERE trip_start_time_millis BETWEEN {start_time_of_nth_segment(segment_metadata_2, 0)} AND {int(start_time_of_nth_segment(segment_metadata_2, 3))-1}
    GROUP BY driver_name
    ORDER BY rating desc
    LIMIT 5"""

query_result = query_sql(query_for_trips_segmentation_2)
display(query_result_to_dataframe(query_result))
display(extract_query_statistics_from_result_dataframe(query_result))

query: 
    SELECT driver_name, avg(driver_rating) AS rating
    FROM trips_segmentation_2
    WHERE trip_start_time_millis BETWEEN 1618159486495 AND 1618309433532
    GROUP BY driver_name
    ORDER BY rating desc
    LIMIT 5


,driver_name,rating
0,Jeannine Robison,2.947761
1,Jason Phillips,2.945946
2,Mindy Smith,2.926667
3,Alfred Obrien,2.920635
4,Jaime Adams,2.890411


,value
numConsumingSegmentsQueried,1
numDocsScanned,148256
numEntriesScannedInFilter,100000
numEntriesScannedPostFilter,296512
numGroupsLimitReached,False
numSegmentsMatched,4
numSegmentsProcessed,4
numSegmentsQueried,7
numServersQueried,3
numServersResponded,3


## Query Routing

In order to efficiently distribute queries across the fleet of servers, brokers maintain so called routing tables, which contain mappings between segments of a table and servers where they are hosted on. 
In case of replicated segments (like in the second table), the routing table contains entries for all servers hosting a single segment. When queries arrive at the broker, the routing tables and segment metadata allow to efficiently scatter queries across servers to balance load across the cluster.

In [24]:
# some helpers for the upcoming examples
def routing_table_for_query(query):
    print("query: " + query)
    return requests.get('http://pinot-broker.pinot:8099/debug/routingTable/sql', params={
        "query" : query
    }).json()

def routing_table_for_table(table):
    return requests.get(f'http://pinot-broker.pinot:8099/debug/routingTable/{table}').json()

def external_view_for_table(table):
    return requests.get(f'http://pinot-controller.pinot:9000/tables/{table}/externalview').json()

def routing_table_for_query_dataframe(query):
    rt = routing_table_for_query(query)
    rt_data = {}

    for server, server_segments in rt.items():
        server_name = server_name_from_instance(server)
        for s in server_segments:
            rt_data[s] = server_name

    rt_data_list = []
    for segment, server in rt_data.items():
        rt_data_list.append({"segment": segment, "server": server})

    rt_data_list.sort(key=lambda L: (int(ordinal_pattern.search(L["segment"]).group(1)), L))
    return pd.DataFrame(rt_data_list)

def routing_table_for_table_dataframe(table):
    rt = routing_table_for_table(table)
    rt_data = {}

    for table_name_type, table_rt in rt.items():
        table_type = re.search('REALTIME|OFFLINE', table_name_type).group()
        for server, server_segments in table_rt.items():
            server_name = server_name_from_instance(server)
            for s in server_segments:
                try:
                    rt_data[s][table_type] = server_name
                except KeyError:
                    rt_data[s] = {table_type: server_name}

    rt_data_list = []
    for segment, type_server in rt_data.items():
        segment_data = {"segment": segment}
        for table_type, server in type_server.items():
            segment_data[table_type] = server
        rt_data_list.append(segment_data)

    rt_data_list.sort(key=lambda L: (int(ordinal_pattern.search(L["segment"]).group(1)), L))
    return pd.DataFrame(rt_data_list)

def external_view_for_table_dataframe(table):
    ev = external_view_for_table(table)
    ev_data = {}

    for table_type, ev_per_type in ev.items():
        if ev_per_type == None:
            continue
        
        for segment, segment_servers in ev_per_type.items():
            if not segment in ev_data:
                ev_data[segment] = {}
            for server, state in segment_servers.items():
                server_name = server_name_from_instance(server)
                try:
                    ev_data[segment][table_type].append(server_name)
                except KeyError:
                    ev_data[segment][table_type] = [server_name]

    return pd.DataFrame(ev_data).transpose()

First, let's take a look at the external view for both tables. The external view shows an overview, which segments are available on which server. In case of the first table, each segment is only available on a single server. The second table has a replica of each segment on every server.

In [25]:
display(external_view_for_table_dataframe("trips_segmentation_1"))
display(external_view_for_table_dataframe("trips_segmentation_2"))

,REALTIME
trips_segmentation_1__0__0__20210411T1645Z,[pinot-server-1]
trips_segmentation_1__0__1__20210411T1646Z,[pinot-server-1]
trips_segmentation_1__0__2__20210411T1646Z,[pinot-server-1]
trips_segmentation_1__0__3__20210411T1646Z,[pinot-server-1]


,REALTIME
trips_segmentation_2__0__0__20210411T1645Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__1__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__2__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__3__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__4__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__5__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__6__20210411T1646Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"


We can use the broker's debug endpoint to retrieve a routing table for a specific SQL query. This can be seen as a query execution plan for segments distributed across multiple servers. Similar to calculating an efficient query execution plan in classical RDBMSs, Pinot takes a look at metadata, statistics and server associations.
The routing table might change everytime an identical query is executed, as brokers try to distribute compute load across servers hosting the same segment.

In [26]:
routing_table_for_query_dataframe(query_for_trips_segmentation_1.replace("trips_segmentation_1", "trips_segmentation_1_REALTIME"))

query: 
    SELECT driver_name, sum(count) AS trips_count
    FROM trips_segmentation_1_REALTIME
    WHERE trip_start_time_millis BETWEEN 1618159486495 AND 1618239458972
    GROUP BY driver_name
    ORDER BY trips_count desc
    LIMIT 5


,segment,server
0,trips_segmentation_1__0__0__20210411T1645Z,pinot-server-1
1,trips_segmentation_1__0__1__20210411T1646Z,pinot-server-1
2,trips_segmentation_1__0__2__20210411T1646Z,pinot-server-1
3,trips_segmentation_1__0__3__20210411T1646Z,pinot-server-1


For the second query, the routing table shows, that the broker will try to equally distribute load between all the servers, as the segments are replicated.

In [27]:
routing_table_for_query_dataframe(query_for_trips_segmentation_2.replace("trips_segmentation_2", "trips_segmentation_2_REALTIME"))

query: 
    SELECT driver_name, avg(driver_rating) AS rating
    FROM trips_segmentation_2_REALTIME
    WHERE trip_start_time_millis BETWEEN 1618159486495 AND 1618309433532
    GROUP BY driver_name
    ORDER BY rating desc
    LIMIT 5


,segment,server
0,trips_segmentation_2__0__0__20210411T1645Z,pinot-server-1
1,trips_segmentation_2__0__1__20210411T1646Z,pinot-server-2
2,trips_segmentation_2__0__2__20210411T1646Z,pinot-server-0
3,trips_segmentation_2__0__3__20210411T1646Z,pinot-server-1
4,trips_segmentation_2__0__4__20210411T1646Z,pinot-server-2
5,trips_segmentation_2__0__5__20210411T1646Z,pinot-server-0
6,trips_segmentation_2__0__6__20210411T1646Z,pinot-server-1


## Advanced Configuration

The presented tables are rather simple and just demonstrate the basic mechanisms of segmentation, replication and query routing in Pinot. However, Pinot offers much more advanced configuration options for tweaking segment replication, availability and placement in large-scale Pinot clusters.

For example, Pinot servers can be grouped in so called "replica groups", that can be spread across different availability zones. Segment replicas will then be assigned to servers in different replica groups in order to achieve high-availability setups. Furthermore, segments can be partitioned based on column values to further increase query performance by decreasing the number of segments that need to be processed for a given query. This is very similar to partitioning/sharding in typical RDBMSs.
Additionally, servers can be assigned to different tenants for sharing a cluster across teams or grouped into server-pools to achieve no-downtime rolling restarts of large clusters.

All of these options show, that segmentation in Pinot is in the simplest aspects quite comparable to sharding mechanism in other database systems, but it is also much more advanced to support large-scale analytical use-cases while maintaining high performance.

# Batch Ingestion and Hybrid Tables

As mentioned earlier, Pinot also support ingesting data from batch processing jobs. For offline tables, the same principles apply as for realtime tables with regards to segmentation and query routing. 
Though, segments are compiled and packaged outside Pinot. For this purpose, Pinot offers different mechanisms to load pre-built segments from object stores (such as S3) or HDFS or to build new segments using Hadoop and/or Spark.
Segments are packaged as gzipped tar-archives (including data, index maps, column statistics) and can be uploaded to and downloaded from the controller.

While offline tables can be used standalone similar to the realtime tables presented above, a more interesting option is to combine an offline and a realtime table to form a hybrid table.
Hybrid tables are comprised of two individual tables, one offline table and one hybrid table, both sharing the same name, schema and – most importantly – time column. The hybrid table can be queried just like any other table, but the broker will transparently rewrite queries to fetch older records from the offline table and newer records from the realtime table.
This allows to process, deduplicate and sanitize records before pushing them to long-term storage. This is a key differentiator between Pinot and other databases and OLAP stores. It allows Pinot to achieve high-throughput ingestion, low-latency realtime analytics, while still allowing to backfill data in batch processing.

Since version `0.6.0` Pinot also offers a mechanism to regularly move records from a realtime table to the corresponding offline table. To configure this, the user can schedule a task, which should be executed on a minion instance for example once every day. The task execution will then take over downloading, transforming, aggregating, sorting and uploading of segments.

To demonstrate how batch ingestion and hybrid tables work in Pinot without setting up an external batch processing system or periodic segment transformation job, we're going to create a realtime table reading from our Kafka `trips` topic, download completed segments from the controller and re-upload them as offline segments.

First, we need to create both tables (note the shared name and schema):

In [28]:
# common configuration used for both tables types
table_config_template = {
  "tableName": "trips_hybrid",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
    "replication": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "invertedIndexColumns": [
        "rider_name",
        "driver_name",
        "start_location",
        "end_location"
    ]
  },
  "metadata": {
    "customConfigs": {}
  }
}

In [29]:
# create offline table
table_config = copy.deepcopy(table_config_template)
table_config["tableType"] = "OFFLINE"
print(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

{'status': 'Table trips_hybrid_OFFLINE succesfully added'}


In [30]:
# create realtime table
table_config = copy.deepcopy(table_config_template)
table_config["tableType"] = "REALTIME"
table_config["segmentsConfig"]["replicasPerPartition"] = "1"
table_config["tableIndexConfig"]["streamConfigs"] = {
  "streamType": "kafka",
  "stream.kafka.consumer.type": "simple",
  "stream.kafka.topic.name": "trips",
  "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
  "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
  "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
  "stream.kafka.broker.list": "pinot-kafka:9092",
  "realtime.segment.flush.threshold.time": "12h",
  "realtime.segment.flush.threshold.size": "50000",
  "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
}
print(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

{'status': 'Table trips_hybrid_REALTIME succesfully added'}


Let's again wait for our table to finish loading the data from Kafka.

In [31]:
wait_for_table_to_finish_loading("trips_hybrid")

waiting for table trips_hybrid to finish loading (loaded 64873 docs)
waiting for table trips_hybrid to finish loading (loaded 274981 docs)
waiting for table trips_hybrid to finish loading (loaded 309911 docs)
--Consumption of generated data for table trips_hybrid finished, (loaded 309911 docs)--


Let's take a look at the external view of the hybrid table before touching it. We can see some realtime segments, that were built from the data stream from Kafka, but there are no offline segments so far:

In [32]:
external_view_for_table_dataframe("trips_hybrid")

,REALTIME
trips_hybrid__0__0__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__1__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__2__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__3__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__4__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__5__20210411T1650Z,[pinot-server-1]
trips_hybrid__0__6__20210411T1650Z,[pinot-server-1]


In [33]:
# helpers for transforming realtime segments to offline segment
tmp_hybrid_basedir = "/tmp/trips_hybrid"
# cleanup old artifacts if any
shutil.rmtree(tmp_hybrid_basedir, ignore_errors=True)
os.mkdir(tmp_hybrid_basedir)

def path_for_realtime_tar(segment_name):
    return f"{tmp_hybrid_basedir}/{segment_name}.tar.gz"

def path_for_offline_dir(segment_name):
    return f"{tmp_hybrid_basedir}/{segment_name}_offline"

def path_for_offline_tar(segment_name):
    return f"{tmp_hybrid_basedir}/{segment_name}_offline.tar.gz"

def download_segment(segment_metadata):
    segment_name = segment_metadata["segment.name"]
    download_url = segment_metadata["segment.realtime.download.url"]
    segment_realtime_tar = path_for_realtime_tar(segment_name)

    # cleanup old downloads
    try:
        os.remove(segment_realtime_tar)
    except OSError:
        pass

    # download realtime segment tar
    response = requests.get(download_url, stream=True)
    with open(segment_realtime_tar, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    
    print(f"segment {segment_name} downloaded from {download_url} to {segment_realtime_tar}")
    return segment_realtime_tar

def untar_segment(segment_metadata):
    segment_name = segment_metadata["segment.name"]
    segment_offline_basedir = path_for_offline_dir(segment_name)
    segment_realtime_tar = path_for_realtime_tar(segment_name)

    # cleanup old artifacts if any
    shutil.rmtree(segment_offline_basedir, ignore_errors=True)

    # extract downloaded segment tar
    with tarfile.open(segment_realtime_tar, 'r:gz') as tar:
        tar.extractall(path=segment_offline_basedir)

    print(f"segment {segment_name} untarred to {segment_offline_basedir}")
    return segment_offline_basedir

def transform_segment(segment_metadata):
    realtime_table_name = segment_metadata["segment.table.name"]
    offline_table_name = realtime_table_name.replace("REALTIME", "OFFLINE")
    segment_name = segment_metadata["segment.name"]
    segment_offline_basedir = path_for_offline_dir(segment_name)
    
    # modify metadata.properties of segment
    segment_offline_dir = segment_offline_basedir + "/" + segment_name
    metadata_file = segment_offline_dir + "/v3/metadata.properties"
    metadata_contents = None
    with open(metadata_file, 'r') as file:
      metadata_contents = file.read()
    
    metadata_contents = metadata_contents.replace(realtime_table_name, offline_table_name)
    
    with open(metadata_file, 'w') as file:
      file.write(metadata_contents)
    del metadata_contents

    # create new offline segment tar
    segment_offline_tar = path_for_offline_tar(segment_name)
    with tarfile.open(segment_offline_tar, 'w:gz') as tar:
        tar.add(segment_offline_dir, arcname=segment_name)

    print(f"segment {segment_name} transformed to offline segment to {segment_offline_tar}")
    return segment_offline_tar

def upload_segment_to_offline_table(segment_metadata):
    realtime_table_name = segment_metadata["segment.table.name"]
    segment_name = segment_metadata["segment.name"]
    segment_offline_tar = path_for_offline_tar(segment_name)
    table_name = realtime_table_name.replace("_REALTIME", "_OFFLINE")
    
    # POST segment as multipart/form-data for key 'segment'
    with open(segment_offline_tar, 'rb') as tar:
        response = requests.post(f'http://pinot-controller.pinot:9000/v2/segments?table={table_name}', files={
            'segment': tar
        })
        print(response)
        print(response.json())

def transform_and_upload_nth_segment_to_offline_table(segment_metadata, n):
    nth_meta = segment_metadata_of_nth_segment(segment_metadata, n, table_type="REALTIME")
    
    # download, transform and upload all in one row
    download_segment(nth_meta)
    untar_segment(nth_meta)
    transform_segment(nth_meta)
    upload_segment_to_offline_table(nth_meta)

Now, we fetch the first two segments from the controller, manipulate the metadata and re-upload them to the controller as offline segments:

In [34]:
segment_metadata_hybrid = segment_metadata_for_table("trips_hybrid")

transform_and_upload_nth_segment_to_offline_table(segment_metadata_hybrid, 0)
transform_and_upload_nth_segment_to_offline_table(segment_metadata_hybrid, 1)

segment trips_hybrid__0__0__20210411T1650Z downloaded from http://pinot-controller-0.pinot-controller-headless.pinot.svc.cluster.local:9000/segments/trips_hybrid/trips_hybrid__0__0__20210411T1650Z to /tmp/trips_hybrid/trips_hybrid__0__0__20210411T1650Z.tar.gz
segment trips_hybrid__0__0__20210411T1650Z untarred to /tmp/trips_hybrid/trips_hybrid__0__0__20210411T1650Z_offline
segment trips_hybrid__0__0__20210411T1650Z transformed to offline segment to /tmp/trips_hybrid/trips_hybrid__0__0__20210411T1650Z_offline.tar.gz
<Response [200]>
{'status': 'Successfully uploaded segment: trips_hybrid__0__0__20210411T1650Z of table: trips_hybrid_OFFLINE'}
segment trips_hybrid__0__1__20210411T1650Z downloaded from http://pinot-controller-0.pinot-controller-headless.pinot.svc.cluster.local:9000/segments/trips_hybrid/trips_hybrid__0__1__20210411T1650Z to /tmp/trips_hybrid/trips_hybrid__0__1__20210411T1650Z.tar.gz
segment trips_hybrid__0__1__20210411T1650Z untarred to /tmp/trips_hybrid/trips_hybrid__0__1

The external view for our hybrid table now shows the newly added offline segments:

In [35]:
external_view_for_table_dataframe("trips_hybrid")

,OFFLINE,REALTIME
trips_hybrid__0__0__20210411T1650Z,[pinot-server-0],[pinot-server-1]
trips_hybrid__0__1__20210411T1650Z,[pinot-server-1],[pinot-server-1]
trips_hybrid__0__2__20210411T1650Z,NaN,[pinot-server-1]
trips_hybrid__0__3__20210411T1650Z,NaN,[pinot-server-1]
trips_hybrid__0__4__20210411T1650Z,NaN,[pinot-server-1]
trips_hybrid__0__5__20210411T1650Z,NaN,[pinot-server-1]
trips_hybrid__0__6__20210411T1650Z,NaN,[pinot-server-1]


This example query lists the top 5 riders in terms of total trip time. It shows that hybrid tables can be queried in the exact same way, as realtime tables:

In [36]:
query_for_hybrid = """
    SELECT rider_name, sum(trip_end_time_millis - trip_start_time_millis) / (60*60*1000) AS trip_time_sum
    FROM trips_hybrid
    GROUP BY rider_name
    ORDER BY trip_time_sum DESC
    LIMIT 5
    """

query_result = query_sql(query_for_hybrid)
query_result_to_dataframe(query_result)

query: 
    SELECT rider_name, sum(trip_end_time_millis - trip_start_time_millis) / (60*60*1000) AS trip_time_sum
    FROM trips_hybrid
    GROUP BY rider_name
    ORDER BY trip_time_sum DESC
    LIMIT 5
    


,rider_name,trip_time_sum
0,Alice Reese,6035.993905
1,Dennis Smith,5864.333590
2,Rodney Jones,3620.798730
3,Debbie Miller,3590.483984
4,Mary Franco,3545.339102


# Indexing

## Table Creation with Different Indexing Configurations

To demonstrate the different indexing options and mechanisms, that Pinot offers, we will create some tables with different index configurations, describe their key properties and compare query performance.
The index configurations are applied to the `tableIndexConfig`-section the table configuration.

In [37]:
# Array to collect details of created tables
table_list = []

table_config_template = {
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "trips",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "20000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
} 

# helper function
def createTable(newTable_name, index_text, tableconfig_json):
    # Input: Name of new table, index description, table configuration in json structure
    response = requests.post('http://pinot-controller.pinot:9000/tables', json=tableconfig_json)
    print(response)
    print(response.text)
    table_list.append([newTable_name, index_text])


execution_start_time = int(round(time.time() * 1000))

# Create a new table with default index for each column (no configuration required)
newTable_defaultIndex = copy.deepcopy(table_config_template)
newTable_defaultIndex["tableName"] = "trips_default_index"
createTable(newTable_defaultIndex["tableName"], 'Default Index (Dictionary-encoded forward index with bit compression) for each column', newTable_defaultIndex)

# Create a new table with raw value forward index
newTable_rawForwardIndex = copy.deepcopy(table_config_template)
newTable_rawForwardIndex["tableName"] = "trips_rawForwardIndex"
newTable_rawForwardIndex["tableIndexConfig"]["noDictionaryColumns"] = ["start_location"]
createTable(newTable_rawForwardIndex["tableName"], 'Raw value forward index on start_location', newTable_rawForwardIndex)

# Create a new table with sorted forward index with run-length encoding
newTable_sortedForwardIndex = copy.deepcopy(table_config_template)
newTable_sortedForwardIndex["tableName"] = "trips_sortedForwardIndex"
newTable_sortedForwardIndex["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedForwardIndex["tableName"], 'Sorted forward index with run-length encoding on start location', newTable_sortedForwardIndex)

# Create a new table with bitmap inverted index
newTable_bitmapInvertedIndex = copy.deepcopy(table_config_template)
newTable_bitmapInvertedIndex["tableName"] = "trips_bitmapInvertedIndex_startLocation"
newTable_bitmapInvertedIndex["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
createTable(newTable_bitmapInvertedIndex["tableName"], 'Bitmap inverted index on start_location', newTable_bitmapInvertedIndex)

# Create a new table with sorted inverted index
newTable_sortedInvertedIndex = copy.deepcopy(table_config_template)
newTable_sortedInvertedIndex["tableName"] = "trips_sortedInvertedIndex_startLocation"
newTable_sortedInvertedIndex["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
newTable_sortedInvertedIndex["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedInvertedIndex["tableName"], 'Sorted inverted index on start_location', newTable_sortedInvertedIndex)

# Create a new table with star tree index
newTable_starTree = copy.deepcopy(table_config_template)
newTable_starTree["tableName"] = "trips_starTreeIndex"
newTable_starTree["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "rider_is_premium",
      "start_location_state",
      "end_location"
    ],
    "functionColumnPairs": [
      "SUM__payment_amount",
    ],
    "maxLeafRecords": 1
  }]
createTable(newTable_starTree["tableName"], 'Star Tree', newTable_starTree)

# Create a new table with text index
newTable_textIndex = copy.deepcopy(table_config_template)
newTable_textIndex["tableName"] = "trips_textIndex"
newTable_textIndex["fieldConfigList"]= [
  {
     "name":"driver_name",
     "encodingType":"RAW",
     "indexType":"TEXT"
  },
  {
     "name":"rider_name",
     "encodingType":"RAW",
     "indexType":"TEXT"
  }
]
newTable_textIndex["tableIndexConfig"]["noDictionaryColumns"] = [
     "driver_name",
     "rider_name"
 ]
createTable(newTable_textIndex["tableName"], 'Text Index', newTable_textIndex)

<Response [200]>
{"status":"Table trips_default_index_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_rawForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_bitmapInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_textIndex_REALTIME succesfully added"}


All created tables consume data from the same Kafka topic (`trips`). Therefore, all of them will contain the same data records.
To ensure, that the consumption of the tables has finished before executing queries, we use the helper function from above to wait until the tables have finished loading.

In [38]:
# star-tree index building takes some time (longer than other tables), wait for it first
wait_for_table_to_finish_loading("trips_starTreeIndex")

for table in table_list:
    table_name = table[0]
    if table_name == "trips_starTreeIndex":
        continue
    wait_for_table_to_finish_loading(table_name, wait_time=2)

waiting for table trips_starTreeIndex to finish loading (loaded 20000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 60000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 87123 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 120000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 160000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 200000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 240000 docs)
waiting for table trips_starTreeIndex to finish loading (loaded 309911 docs)
--Consumption of generated data for table trips_starTreeIndex finished, (loaded 309911 docs)--
waiting for table trips_default_index to finish loading (loaded 309911 docs)
--Consumption of generated data for table trips_default_index finished, (loaded 309911 docs)--
waiting for table trips_rawForwardIndex to finish loading (loaded 309911 docs)
--Consumption of generated data for table

In [39]:
# Read first start location and first driver name to use it as variables in upcoming queries
query_result = query_sql(f"SELECT start_location, driver_name FROM trips_default_index LIMIT 1")

startLocation = query_result['resultTable']['rows'][0][0]
# Read the first name of the driver name
driverName = (query_result['resultTable']['rows'][0][1]).split()[0]       

print(f"Using '{startLocation}' as start location and '{driverName}' as driver name for upcoming queries.")

query: SELECT start_location, driver_name FROM trips_default_index LIMIT 1
Using 'Emden' as start location and 'Michael' as driver name for upcoming queries.


## Table Size

Although all tables contain the same amount of records and also the same record values, the table size differs.
This is because of the different indexes used. E.g. a text index on two columns consumes much more space compared to the raw forward index. The Star-Tree index is allocating the most disk space, as Pinot materializes pre-aggregations for calculations on defined metric columns.

In [40]:
table_size_data = {}

for table in table_list:
    response = requests.get(f'http://pinot-controller.pinot:9000/tables/{table[0]}_REALTIME/size?detailed=false').json()
    table_name = response['tableName']
    table_size_data[table_name] = {"description": table[1], "reportedSizeInMB": response['reportedSizeInBytes']/1024/1024}

pd.set_option('max_colwidth', 400)
display(pd.DataFrame(table_size_data).transpose())
pd.reset_option('max_colwidth')

,description,reportedSizeInMB
trips_default_index_REALTIME,Default Index (Dictionary-encoded forward index with bit compression) for each column,22.197715
trips_rawForwardIndex_REALTIME,Raw value forward index on start_location,24.548973
trips_sortedForwardIndex_REALTIME,Sorted forward index with run-length encoding on start location,20.196658
trips_bitmapInvertedIndex_startLocation_REALTIME,Bitmap inverted index on start_location,23.044106
trips_sortedInvertedIndex_startLocation_REALTIME,Sorted inverted index on start_location,20.197087
trips_starTreeIndex_REALTIME,Star Tree,39.42692
trips_textIndex_REALTIME,Text Index,36.26193


## Comparison of Indexing Options

The function `executeSQLStatement` takes a query string and an array containing table names and index descriptions as input parameters. It executes the query on all tables which are defined in the array `table_list`. If `specific_tables_array` is empty, the query will be executed on all tables which have been created for this chapter. The top two records of the result data set are displayed once to get an insight into the result. Additionally, the function will create a `DataFrame` listing query execution statistics for each table. Metrics of one query execution will only be appended to the `DataFrame` if no exception occurs.

In [41]:
def executeSQLStatement(sql_statement_with_variable, specific_tables_array):
    pd.set_option('display.max_colwidth', None)
    df_metrics = pd.DataFrame(columns=['indextype','table', 'numDocsScanned',
       'numEntriesScannedInFilter', 'numEntriesScannedPostFilter',
       'totalDocs', 'timeUsedMs',
       'minConsumingFreshnessTimeMs',
       'exceptions'])
    b_resultRecordsNotShown = True;
    if not specific_tables_array:
        table_list_statement = table_list
    else:
        table_list_statement = specific_tables_array 
    for table in table_list_statement:
    
        sql_statement = sql_statement_with_variable.replace("XX_TABLE",table[0])
        sql_statement = sql_statement.replace("XX_STARTLOCATION","'"+startLocation+"'")
        sql_statement = sql_statement.replace("XX_DRIVERNAME","'"+driverName+"'") 
        response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : sql_statement
        })
        response_json=response.json()
        d = {'indextype': table[1], 'table': table[0],'numDocsScanned': [response_json['numDocsScanned']],'numDocsScanned': [response_json['numDocsScanned']],'numEntriesScannedInFilter': [response_json['numEntriesScannedInFilter']], 'numEntriesScannedPostFilter':[response_json['numEntriesScannedPostFilter']],'totalDocs':[response_json['totalDocs']],'timeUsedMs':[response_json['timeUsedMs']],'minConsumingFreshnessTimeMs':[response_json['minConsumingFreshnessTimeMs']],'exceptions':[response_json['exceptions']]}
        df_metrics_new = pd.DataFrame(data=d)
        if not response_json['exceptions']:
             df_metrics = df_metrics.append(df_metrics_new,ignore_index=True)
       

        if b_resultRecordsNotShown:
            try:
                if not response_json['exceptions']:
                    columnNames = response_json['resultTable']['dataSchema']['columnNames']
                    rows = response_json['resultTable']['rows']

                    result_dataframe = pd.DataFrame(columns=columnNames,data=rows)
                    print("Top two result records of: " + sql_statement)
                    display(result_dataframe.head(2))
                    b_resultRecordsNotShown = False
            except:
                pass

    display(df_metrics)

### Metrics

The main metrics of a query execution we will check are:
- __timeUsedMs__: Total time between broker receiving the query request request and sending the response back to the client.
- __numDocScanned__: Number of documents/records scanned while query processing. (Includes records scanned in the filter phase as well as after applying the filter.)
- __numEntriesScannedInFilter__: It is an indicator of the latency contributed by the lookup phase. If this number is high, applying an index on the selection criteria might improve performance, especially if the selection criteria is highly selective.
- __numEntriesScannedPostFilter__: High number is an indicator for low selectivity. Instead of regular indices, a star-tree index could help.

### Index Types

For the tables create above, we configured the following index types:

__Forward Index__
- __Default Index: Dictionary-encoded forward index with bit compression__: 
    Apache Pinot will use this index by default for each column if no other index is configured in the table configuration. An id is assigned to each distinct value of the column, afterwards a dictionary is built matching an id to the value. In the forward index, only the bit-compressed id is persisted instead of the values. This compression improves space efficiency of the storage, if there are only a few distinct values.

<img src="https://raw.githubusercontent.com/timebertt/adm-pinot/master/images/MarkdownTable_DefaultIndex.png" width="50%" height="50%">

- __Raw Value Forward Index__: A raw value forward index is configured as a `noDictionaryColumn` in the table configuration. Instead of dictionary ids, the raw values will be stored in columns. Because of that, no dictionary lookup is required and due to the locality of values the performance of scanning large number of values is improved. 
- __Sorted forward index with run-length encoding__: The sorted forward index is applied on top of the dictionary-encoding. For each dictionary id, a start and end document id is stored. Only one sorted column can be configured per table.
   
__Inverted Index__: Inverted Indexes reduce the number of records which need be processed by identifying the ones which contain the search term. The inverted index is created by selecting all distinct values of a given column. For each value, a list of document ids which contain the value will be stored. If we search e.g. for "Hessen" as a state, we can look up the inverted index for "Hessen" and identify the documents in which that value appears. 
- __Bitmap inverted index__: A map from each value to a bitmap is maintained for the column which is enabled as bitmap inverted index (e.g. "Thüringen" -> `Doc5, Doc1`). If a column is used frequently for filtering, an inverted index will improve the performance. 

<img src="https://raw.githubusercontent.com/timebertt/adm-pinot/master/images/MarkdownTable_BitmapInvertedIndex.png" width="35%" height="50%">

- __Sorted inverted index__: A sorted index can benefit from data locality, but can only be applied to one column.

<img src="https://raw.githubusercontent.com/timebertt/adm-pinot/master/images/MarkdownTable_SortedInvertedIndex.png" width="50%" height="50%">

- __Star-Tree Index__: This index is built on multiple columns and pre-aggregates results per configured dimension hierarchy level, so that less values need to be processed. This can significantly improve query performance for hierarchical data (e.g. groups of users, workspaces, or states of locations in the `trips` example), on the other hand pre-aggregation requires also more disk space (table size can easily grow about twice of the size as the other tables).
- __Text Index__: Text Indexes in Pinot allow to do aribtrary search on `STRING` columns (full text search).

#### Default Index (Dictionary-encoded forward index with bit compression) vs Raw value forward index

In [42]:
executeSQLStatement("select count, driver_name, driver_rating, end_location, end_location_state from XX_TABLE WHERE start_location=XX_STARTLOCATION LIMIT 10000",[['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_rawForwardIndex', 'Raw value forward index on start_location']])

Top two result records of: select count, driver_name, driver_rating, end_location, end_location_state from trips_default_index WHERE start_location='Emden' LIMIT 10000


,count,driver_name,driver_rating,end_location,end_location_state
0,1,Michael Weston,4,Laucha,Thüringen
1,1,Courtney Gonzalez,1,Gummersbach,Nordrhein-Westfalen


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,316,309911,1580,309911,96,1618160041688,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,316,309911,1580,309911,163,1618160038818,[]


The query execution on table `trips_rawForwardIndex` takes more time. The main difference between the two index types is, that the index on column `start_location` of `trips_default_index` creates a dictionary. This dictionary provides compression when values of the columns occurr repeatedly. 
A dictionary index can't provide this advantage over the other index, if the column values have a high cardinality.

#### Default Index (Dictionary-encoded forward index with bit compression) vs Sorted forward index with run-length encoding

In [43]:
executeSQLStatement("select * from XX_TABLE WHERE start_location=XX_STARTLOCATION LIMIT 10000", [['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_sortedForwardIndex', 'Sorted forward index with run-length encoding on start location']])

Top two result records of: select * from trips_default_index WHERE start_location='Emden' LIMIT 10000


,count,driver_name,driver_rating,end_location,end_location_state,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_location_state,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,Michael Weston,4,Laucha,Thüringen,99880,CT-SC-79,906.81,25.0,1618439397147,1,Jeremiah Santana,5,Emden,Niedersachsen,26725,1618473364124,1618440571397,1174250
1,1,Courtney Gonzalez,1,Gummersbach,Nordrhein-Westfalen,51643,MY-MD-47,270.84,5.0,1618440921125,1,Patricia Stone,1,Emden,Niedersachsen,26725,1618455371860,1618443393678,2472553


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,316,309911,6004,309911,65,1618160041688,[]
1,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,316,0,6004,309911,51,1618160039711,[]


The sorted forward index on column `start_location` of table `trips_sortedForwardIndex` benefits from data locality. Because of this, `numEntriesScannedInFilter` is less than for the column with default index.
Thus, query executions can be faster when using the sorted forward index on column `start_location`.

#### Default Index (Dictionary-encoded forward index with bit compression) vs Inverted index (Bitmap + Sorted)

In [45]:
executeSQLStatement("select driver_name, rider_name from XX_TABLE WHERE start_location=XX_STARTLOCATION LIMIT 10000", [['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_bitmapInvertedIndex_startLocation', 'Bitmap inverted index on start_location'],['trips_sortedInvertedIndex_startLocation','Sorted inverted index on start_location']])

Top two result records of: select driver_name, rider_name from trips_default_index WHERE start_location='Emden' LIMIT 10000


,driver_name,rider_name
0,Michael Weston,Jeremiah Santana
1,Courtney Gonzalez,Patricia Stone


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,316,309911,632,309911,66,1618160041688,[]
1,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,316,0,632,309911,22,1618160041841,[]
2,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,316,0,632,309911,13,1618160040531,[]


As we can see, an inverted index can improve the query performance. In this case, no entries have to be scanned in the filtering phase and the query execution time is faster compared to using the dictionary encoded index.
By using the sorted inverted index, the performance can benefit from data locality. 

#### Text Index

A query searching selecting drivers by first name can only be executed successfully on table `trips_textIndex`, as it has a text index defined on column `driver_name`. The same query execution on other tables will fail, the metrics table only displays the executions without an exception.

In [46]:
executeSQLStatement("select * from XX_TABLE WHERE TEXT_MATCH ('driver_name',XX_DRIVERNAME) LIMIT 10000", [['trips_textIndex','Text Index']])

Top two result records of: select * from trips_textIndex WHERE TEXT_MATCH ('driver_name','Michael') LIMIT 10000


,count,driver_name,driver_rating,end_location,end_location_state,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_location_state,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,Michael Waldren,3,Niederstotzingen,Baden-Württemberg,89168,VS-GN-23,598.32,28.0,1618359470089,0,Rhonda Lopez,2,Frankfurt am Main,Hessen,60594,1618396816222,1618361201936,1731847
1,1,Michael Jankowski,4,Hartenstein Zschocken,Sachsen,08118,AW-NK-98,186.39,10.0,1618359495055,1,Clyde Bingham,1,Biesendahlshof,Brandenburg,16306,1618371437097,1618360823304,1328249


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Text Index,trips_textIndex,4742,0,90098,309911,165,1618160050199,[]


#### Star-Tree Index

The Start-Tree index utilizes pre-aggregation of results and is built on multiple columns. This index can improve the performance for specific queries, because the number of values to be processed is reduced by the pre-aggregation. Although usage of a Star-Tree index has the advantage of decreased query runtime, the table size on disk is significantly increased.
For table `trips_starTreeIndex`, a Star-Tree index is built on the dimensions `rider_is_premium`, `start_location_state` and `end_location`. The sum of `payment_amount` is pre-aggregated and materialized based on the configured dimensions.

In [47]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX_TABLE",[["trips_default_index","Default Index (Dictionary-encoded forward index with bit compression) for each column"],["trips_starTreeIndex", "Start Tree"]] )

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index


,sum(payment_amount)
0,2.180779e+08


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,309911,0,309911,309911,40,1618160041688,[]
1,Start Tree,trips_starTreeIndex,9926,0,9926,309911,21,1618160046884,[]


When selecting the Star-Node without grouping by any dimension, Pinot doesn't need to access all documents. Instead, only the Star-Node of each segment is required. The reason, why `numDocsScanned` is not equal to the number of segments is, that there is always one segment that isn't completed yet. Pinot accesses each record of the consuming segment (status `IN_PROGRESS`).

In [48]:
print("The table with the Star Tree Index consists of " + str(len(requests.get('http://pinot-controller.pinot:9000/segments/trips_starTreeIndex').json()[0]['REALTIME'])) + " segments")

The table with the Star Tree Index consists of 16 segments


Filtering on the dimension `rider_is_premium`, which builds the first node of the Star-Tree index, halves the number of `numDocsScanned`. This is because `rider_is_premiumn` is assigned randomly in our data generation, so there is a fifty percent chance that a rider is not premium and less documents of the consuming segment need to be scanned.

In [50]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX_TABLE WHERE rider_is_premium = 0",[["trips_default_index","Default Index (Dictionary-encoded forward index with bit compression) for each column"],["trips_starTreeIndex", "Start Tree"]])

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0


,sum(payment_amount)
0,1.090266e+08


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,154900,309911,154900,309911,35,1618160041688,[]
1,Start Tree,trips_starTreeIndex,4950,9911,4950,309911,17,1618160046884,[]


##### Trace Details For Star-Tree Index

The trace details of the query execution display how much time was spent for which operator execution. We extract the operator details of the following query. The query executed on `trips_default_index` requires a lot of Aggregation Operators, as no data is pre-aggregated like it is the case for the table `trips_starTreeIndex`.

In [51]:
def extract_trace_per_operator_from_result(result):
    trace_data_per_operator = {}
    for server, server_trace_json in result["traceInfo"].items():
        server_trace = json.loads(server_trace_json)
        for trace_dict in server_trace:
            for segment, segment_trace in trace_dict.items():
                for segment_trace_element in segment_trace:
                    for operator, operator_time in segment_trace_element.items():
                        try:
                            trace_data_per_operator[operator] += operator_time
                        except KeyError:
                            trace_data_per_operator[operator] = operator_time
    
    trace_data_array = []
    for operator, operator_time in trace_data_per_operator.items():
        trace_data_array.append({"operator": operator.replace(" Time", ""), "time": operator_time})
    return pd.DataFrame(trace_data_array)

display(extract_trace_per_operator_from_result(query_sql("SELECT SUM(payment_amount) FROM trips_starTreeIndex WHERE rider_is_premium = 0")))

display(extract_trace_per_operator_from_result(query_sql("SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0")))

query: SELECT SUM(payment_amount) FROM trips_starTreeIndex WHERE rider_is_premium = 0


,operator,time
0,AggregationOnlyCombineOperator,3
1,InstanceResponseOperator,4
2,BitmapBasedFilterOperator,0
3,StarTreeFilterOperator,0
4,DocIdSetOperator,1
5,ProjectionOperator,1
6,TransformOperator,1
7,AggregationOperator,3
8,ScanBasedFilterOperator,0


query: SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0


,operator,time
0,AggregationOnlyCombineOperator,25
1,InstanceResponseOperator,25
2,ScanBasedFilterOperator,0
3,DocIdSetOperator,11
4,ProjectionOperator,12
5,TransformOperator,12
6,AggregationOperator,24


### Indexes - Comparison with other database technologies

Two categories of the indexing options demonstrated above, that can also be found in traditional databases, are Forward Indexes and Inverted Indexes.
Forward Indexes are frequently used in traditional database technologies as well to improve storage efficiency.
Search Engines most often rely on a inverted index, like for example EleasticSearch.
We saw, that there is a special index to do a fulltext search for records containing a specific string.

We also demonstrated two other indexing techniques, that are typically not offered by traditional database systems: raw value forward index and Star-Tree index.
The raw value forward index doesn't include dictionaries - when aggregating a large number of values, it can take advantage of data locality for scanning.

The Star-Tree Index is an important and special concept for Pinot, because it is utilizes pre-aggregation for group-by queries to achieve low query latencies. It is specifically designed for the analytical use cases which Pinot was built for and makes it a a key differenciator of Pinot.
E.g. Star-Tree Indexes can bring great benefits, if there is the requirement to return data e.g. per user level - like it is the case for the "Who viewed my Profile" application at LinkedIn.

# Closing Remarks

Compared to most other common databases, setting up Pinot is more complicated as multiple different components are part of a Pinot cluster.
The several different components ensure the flexible scale up of the cluster and the high availability, as the system would continue to serve queries also if in the event of one node going down. This is a big advantage, but the different components also add more operational complexity to the whole landscape.

We experienced the advantages of Pinot, e.g. the possibility to easily stream a high number of data records from a Kafka topic into a table with high throughput. Of course, our experience doesn’t represent a use case like how LinkedIn is using Pinot, as we are producing data for the Kafka stream only once - there is no continuous data stream to the Pinot tables. For demonstration purposes, we wait for the data stream to finish loading to ensure, that there are enough data records for our analysis. In a real scenario, you would operate and execute queries while data is ingested.

To summarize, Pinot is not a general-purpose database like for example PostgreSQL, but is rather highly-specialized database technology. It is definitely more complex to operate and understand, but offers great values for the specific use cases it was built for: near-realtime analytics of huge datasets. Especially the segmentation of tables, hybrid table queries and specialized indexing options contribute to Pinot's extreme scalability, flexibility and performance.

# References

- [1] The Apache Software Foundation, "Apache Pinot Docs", 2021. [Online]. Available at: https://docs.pinot.apache.org/. [Accessed: 11-Apr-2021]
- [2] Rogers, Ryan, et al., "LinkedIn's Audience Engagements API: A Privacy Preserving Data Analytics System at Scale", 2020. ArXiv, abs/2002.05839.
- [3] Im, J.-F., et al., "Pinot: Realtime OLAP for 530 Million Users.", 2018. Proceedings
- [4] Jiang, X., "Star-tree index: Powering fast aggregations on Pinot", 2019. [Online]. Available at: https://engineering.linkedin.com/blog/2019/06/star-tree-index--powering-fast-aggregations-on-pinot. [Accessed: 30-Mar-2021]